In [82]:
import re
import pandas as pd
import numpy as np

In [83]:
df = pd.read_excel("data/sample_association-200705.xlsx", 
                   dtype={'patient_id': str, 'sample_id': str, 'sequencing_sample_id': str, 'demux_donor': str})

In [84]:
df.loc[0, 'patient_id'] = 'CL2'
df.loc[1, 'patient_id'] = 'Mix'
df.loc[2, 'patient_id'] = 'Mix'

In [85]:
df['patient_num'] = df['patient_id'].str.replace("INCOV", "").str.replace("^0+", "")

In [86]:
mapping = {
    None: 'healthy',
    'BL': 'baseline',
    'AC': 'acute'
}
df['status'] = df['sample_id'].str.split("-", expand=True)[1].map(mapping)

In [87]:
df['timepoint'] = df['status'].map({'baseline':'1', 'acute':'2'}).astype('str')

In [88]:
def get_sample_name(row):
    if row['patient_id'] == 'CL2':
        return 'CL2'
    elif row['patient_id'] == 'Mix':
        return '_'.join([row['patient_id'], str(row['demux_donor'])])
    elif row['status'] == 'healthy':
        return row['patient_id']
    else:
        return '-'.join([str(row['patient_num']), str(row['timepoint'])])

In [89]:
df.head()

,patient_id,sample_id,sequencing_sample_id,demux_donor,collection_date,ship_date,sequencing_date,notes,patient_num,status,timepoint
0,CL2,A007BG497-003,NaN,NaN,NaT,NaT,NaT,NaN,CL2,NaN,nan
1,Mix,A020BJ335-002,NaN,NaN,NaT,NaT,NaT,NaN,Mix,NaN,nan
2,Mix,A020BJ380-002,NaN,NaN,NaT,NaT,NaT,NaN,Mix,NaN,nan
3,NaN,A055BH094-003,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,nan
4,NaN,A055BH105-007,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,nan


In [90]:
df['sample_name'] = df.apply(get_sample_name, axis=1)

In [91]:
df.head(10)

,patient_id,sample_id,sequencing_sample_id,demux_donor,collection_date,ship_date,sequencing_date,notes,patient_num,status,timepoint,sample_name
0,CL2,A007BG497-003,NaN,NaN,NaT,NaT,NaT,NaN,CL2,NaN,nan,CL2
1,Mix,A020BJ335-002,NaN,NaN,NaT,NaT,NaT,NaN,Mix,NaN,nan,Mix_nan
2,Mix,A020BJ380-002,NaN,NaN,NaT,NaT,NaT,NaN,Mix,NaN,nan,Mix_nan
3,NaN,A055BH094-003,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,nan,nan-nan
4,NaN,A055BH105-007,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,nan,nan-nan
5,NaN,A132BK188-002,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,nan,nan-nan
6,NaN,A137BG437-003,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,nan,nan-nan
7,NaN,A168BH919-003,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,nan,nan-nan
8,NaN,A308BH444-003,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,nan,nan-nan
9,NaN,A313BJ072-009,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,nan,nan-nan


In [92]:
# batch 1
inds = df['sequencing_sample_id'].str.match("CL2|Mix|\d_\d").fillna(False)
df.loc[inds, "sequencing_batch"] = '1'
df[inds]

,patient_id,sample_id,sequencing_sample_id,demux_donor,collection_date,ship_date,sequencing_date,notes,patient_num,status,timepoint,sample_name,sequencing_batch
33,INCOV001,INCOV001-AC,1_2,unpooled,2020-04-02,2020-04-08,2020-04-11,NaN,1,acute,2,1-2,1
34,INCOV001,INCOV001-BL,1_1,unpooled,2020-03-28,2020-04-08,2020-04-11,NaN,1,baseline,1,1-1,1
35,INCOV002,INCOV002-AC,2_2,unpooled,2020-04-02,2020-04-08,2020-04-11,NaN,2,acute,2,2-2,1
36,INCOV002,INCOV002-BL,2_1,unpooled,2020-03-28,2020-04-08,2020-04-11,NaN,2,baseline,1,2-1,1
39,INCOV004,INCOV004-AC,4_2,unpooled,2020-04-03,2020-04-08,2020-04-11,NaN,4,acute,2,4-2,1
40,INCOV004,INCOV004-BL,4_1,unpooled,2020-03-29,2020-04-08,2020-04-11,NaN,4,baseline,1,4-1,1
79,NaN,NaN,CL2,unpooled,NaT,2020-04-08,2020-04-11,NaN,NaN,NaN,nan,nan-nan,1
80,NaN,NaN,Mix,donor0,NaT,2020-04-08,2020-04-11,NaN,NaN,NaN,nan,nan-nan,1
81,NaN,NaN,Mix,donor1,NaT,2020-04-08,2020-04-11,NaN,NaN,NaN,nan,nan-nan,1


In [93]:
# batch 2
inds = df['sequencing_sample_id'].str.match("S\d").fillna(False)
df.loc[inds, "sequencing_batch"] = '2'
df[inds]

,patient_id,sample_id,sequencing_sample_id,demux_donor,collection_date,ship_date,sequencing_date,notes,patient_num,status,timepoint,sample_name,sequencing_batch
37,INCOV003,INCOV003-AC,S6,donor0,2020-04-04,2020-04-15,2020-04-18,NaN,3,acute,2,3-2,2
38,INCOV003,INCOV003-BL,S5,donor0,2020-03-28,2020-04-15,2020-04-18,NaN,3,baseline,1,3-1,2
41,INCOV005,INCOV005-AC,S8,donor0,2020-04-02,2020-04-15,2020-04-18,NaN,5,acute,2,5-2,2
42,INCOV005,INCOV005-BL,S7,donor0,2020-03-29,2020-04-15,2020-04-18,NaN,5,baseline,1,5-1,2
43,INCOV006,INCOV006-AC,S6,donor1,2020-04-04,2020-04-15,2020-04-18,NaN,6,acute,2,6-2,2
44,INCOV006,INCOV006-BL,S5,donor1,2020-03-31,2020-04-15,2020-04-18,NaN,6,baseline,1,6-1,2
45,INCOV007,INCOV007-AC,S8,donor1,2020-04-03,2020-04-15,2020-04-18,NaN,7,acute,2,7-2,2
46,INCOV007,INCOV007-BL,S7,donor1,2020-03-31,2020-04-15,2020-04-18,NaN,7,baseline,1,7-1,2
47,INCOV008,INCOV008-BL,S1,donor1,2020-04-03,2020-04-15,2020-04-18,NaN,8,baseline,1,8-1,2
48,INCOV009,INCOV009-AC,S4,donor1,2020-04-09,2020-04-15,2020-04-18,NaN,9,acute,2,9-2,2


In [94]:
# batch 3
inds = df['sequencing_sample_id'].str.match("A\d").fillna(False)
df.loc[inds, "sequencing_batch"] = '3'
df[inds]

,patient_id,sample_id,sequencing_sample_id,demux_donor,collection_date,ship_date,sequencing_date,notes,patient_num,status,timepoint,sample_name,sequencing_batch
50,INCOV010,INCOV010-AC,A3,donor0,2020-04-21,2020-04-29,2020-05-01,NaN,10,acute,2,10-2,3
56,INCOV013,INCOV013-AC,A5,donor0,2020-04-14,2020-04-29,2020-05-01,NaN,13,acute,2,13-2,3
57,INCOV013,INCOV013-BL,A4,donor1,2020-04-10,2020-04-29,2020-05-01,NaN,13,baseline,1,13-1,3
59,INCOV015,INCOV015-AC,A5,donor1,2020-04-16,NaT,2020-05-01,NaN,15,acute,2,15-2,3
60,INCOV015,INCOV015-BL,A4,donor0,2020-04-12,2020-04-29,2020-05-01,NaN,15,baseline,1,15-1,3
61,INCOV016,INCOV016-AC,A7,donor1,2020-04-15,2020-04-29,2020-05-01,NaN,16,acute,2,16-2,3
62,INCOV016,INCOV016-BL,A6,donor1,2020-04-13,2020-05-06,2020-05-01,NaN,16,baseline,1,16-1,3
63,INCOV017,INCOV017-AC,A8,See note,NaT,NaT,2020-05-01,NaN,17,acute,2,17-2,3
64,INCOV017,INCOV017-BL,A8,See note,NaT,NaT,2020-05-01,NaN,17,baseline,1,17-1,3
69,INCOV022,INCOV022-AC,A7,donor0,NaT,NaT,2020-05-01,NaN,22,acute,2,22-2,3


In [103]:
# The last few ones (BP...) patient id is equal to sample id
# And sample_name must also be adjusted
df.loc[82:, 'sample_id'] = df.loc[82:, 'patient_id']
df.loc[82:, 'sample_name'] = df.loc[82:, 'sample_id']
df.loc[79:82, 'sample_name'] = df.loc[79:82, 'sequencing_sample_id']
df.loc[81, 'sample_name'] = 'Mix_donor1'
df.tail(9)

,patient_id,sample_id,sequencing_sample_id,demux_donor,collection_date,ship_date,sequencing_date,notes,patient_num,status,timepoint,sample_name,sequencing_batch
78,INCOV031,INCOV031-BL,NaN,NaN,NaT,NaT,NaT,NaN,31,baseline,1,31-1,NaN
79,NaN,NaN,CL2,unpooled,NaT,2020-04-08,2020-04-11,NaN,NaN,NaN,nan,CL2,1
80,NaN,NaN,Mix,donor0,NaT,2020-04-08,2020-04-11,NaN,NaN,NaN,nan,Mix,1
81,NaN,NaN,Mix,donor1,NaT,2020-04-08,2020-04-11,NaN,NaN,NaN,nan,Mix_donor1,1
82,BP0219101,BP0219101,A2,donor0,NaT,NaT,2020-05-01,NaN,BP0219101,NaN,nan,A2,3
83,BP0319250,BP0319250,A2,donor1,NaT,NaT,2020-05-01,NaN,BP0319250,NaN,nan,BP0319250,3
84,BP0419769,BP0419769,A3,donor1,NaT,NaT,2020-05-01,NaN,BP0419769,NaN,nan,BP0419769,3
85,BP05191345,BP05191345,A1,donor0,NaT,NaT,2020-05-01,NaN,BP05191345,NaN,nan,BP05191345,3
86,BP08191115,BP08191115,A1,donor1,NaT,NaT,2020-05-01,NaN,BP08191115,NaN,nan,BP08191115,3


In [104]:
df.to_csv("data/sample_association-200705_cleaned.csv", index=False)